In [1]:
using Plots
using Distributions
using Random, DataFrames
using LinearAlgebra
using MLJ, MLJLinearModels
include("src/qsrfe.jl")
using .qsrfe

In [2]:
#point cloud
n, m = 20, 50
function ontoring(data, radius)
    da = zeros(size(data))
    for i in 1:size(data,1)
        da[i,:] = data[i,:] ./ norm(data[i,:]) .* radius
    end
    return da
end
data1 = rand(Normal(0.,0.5),(n,2))
data2 = ontoring(rand(Normal(0.,0.5),(m,2)),4) .+ rand(Normal(0.,0.5),(m,2))
xtrain = vcat(data1,data2)
xtrain = vcat(xtrain, xtrain .+ [7 5],xtrain .+ [-1 -7])
ytrain = [ones(Float64,n) .* 1; -ones(Float64,m)];
ytrain = vcat(ytrain, ytrain, ytrain);

#100 times 100 grid hack
x = range(-13, 13, length=100)
y = range(-13, 13, length=100)
z = [[i j] for i in x, j in y]
zz = zeros(Float64,(1,2))
for i in 1:100, j in 1:100
    zz = vcat(zz,z[i,j])
end
zz = zz[2:size(zz,1),:];

In [7]:
#prediction contour plot

srfe = qsrfe.srfeRegressor(N=1000,σ2=1.0, λ= 0.0003, intercept=false)

c, ω, ζ = qsrfe.fit(srfe,xtrain,ytrain; verbose= true)
zzz = qsrfe.predict(srfe,zz,c,ω,ζ)
zzz = reshape(zzz,(100,100));

support: 129/1000

In [9]:

contour(x,y,zzz, fmt = :svg)
title!("#active-features = 129")
scatter!(xtrain[ytrain .== 1,1],xtrain[ytrain .== 1,2], label="y = 1")
scatter!(xtrain[ytrain .== -1,1],xtrain[ytrain .== -1,2],label="y = -1")
savefig("active129.svg")

"c:\\Users\\grsbe\\Desktop\\Masterthesis\\qsrfe\\active129.svg"

In [21]:
#prediction contour plot, Quantized version
quantizer = qsrfe.βQ(K=2,λ=32,condense=true)
srfe = qsrfe.srfeRegressor(N=2^12, σ2=1., λ= 0.01, intercept=false)

c, ω, ζ = qsrfe.fit(srfe,xtrain,ytrain,quantizer, verbose=true)
#c, ω, ζ = qsrfe.fit(srfe,xtrain,ytrain; verbose= true)
zzz = qsrfe.predict(srfe,zz,c,ω,ζ,quantizer)
#zzz = qsrfe.predict(srfe,zz,c,ω,ζ)
zzz = reshape(zzz,(100,100));

support: 14/128